In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sentencepiece as spm
import math
import re
from tqdm.notebook import tqdm
import random
from torch.nn.utils.rnn import pad_sequence

In [2]:
kor_path = "/Users/jian_lee/Desktop/aiffel/data/transformer/korean-english-park.train/korean-english-park.train.ko"
eng_path = "/Users/jian_lee/Desktop/aiffel/data/transformer/korean-english-park.train/korean-english-park.train.en"

In [3]:
# 데이터 정제 및 토큰화
def clean_corpus(kor_path, eng_path):
    with open(kor_path, "r", encoding="utf-8") as f:
        kor = f.read().splitlines()
    with open(eng_path, "r", encoding="utf-8") as f:
        eng = f.read().splitlines()
    
    assert len(kor) == len(eng), "Korean and English corpus sizes do not match!"
    
    # 중복 제거 (병렬 데이터를 튜플로 묶어 처리)
    unique_pairs = set(zip(kor, eng))
    
    # 리스트로 변환
    cleaned_corpus = list(unique_pairs)
    
    return cleaned_corpus

cleaned_corpus = clean_corpus(kor_path, eng_path)

In [4]:
# 정규식 패턴을 사전 컴파일하여 속도 최적화
TOKEN_CLEANER = re.compile(r"[^a-zA-Zㄱ-ㅎ가-힣.,!? ]")
PUNCTUATION_SPACING = re.compile(r"([.,!?])")
MULTI_SPACE_CLEANER = re.compile(r"\s+")

def preprocess_sentence(sentence):
    """
    문장을 정제하는 함수
    1. 소문자로 변환
    2. 불필요한 문자 제거
    3. 문장부호 양옆에 공백 추가
    4. 연속된 공백을 하나로 변환
    """
    sentence = sentence.lower() # 소문자 변환
    sentence = TOKEN_CLEANER.sub("", sentence)  # 불필요한 문자 제거
    sentence = PUNCTUATION_SPACING.sub(r" \1 ", sentence)  # 문장부호 공백 추가
    sentence = MULTI_SPACE_CLEANER.sub(" ", sentence)  # 다중 공백 제거
    sentence = sentence.strip()  # 앞뒤 공백 제거
    return sentence

In [5]:
def generate_tokenizer(corpus, vocab_size=20000, lang="ko", pad_id=0, bos_id=1, eos_id=2, unk_id=3):
    """
    SentencePiece 토크나이저 학습 및 로드
    - corpus: 학습할 말뭉치 (리스트 형태)
    - vocab_size: 단어 사전 크기
    - lang: 언어 식별자 (ko/en 등)
    - 특수 토큰: PAD, BOS, EOS, UNK 지정 가능
    """
    
    # 임시 파일 없이 메모리에서 바로 학습할 수도 있지만, 여기서는 기존 방식 유지
    temp_file = f"{lang}_corpus.txt"
    with open(temp_file, "w", encoding="utf-8") as f:
        for sentence in corpus:
            f.write(sentence + "\n")
    
    # SentencePiece 모델 학습
    spm.SentencePieceTrainer.train(
        input=temp_file,
        model_prefix=lang,
        vocab_size=vocab_size,
        pad_id=pad_id,
        bos_id=bos_id,
        eos_id=eos_id,
        unk_id=unk_id,
        model_type="unigram"
    )
    
    # 학습된 모델 로드
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f"{lang}.model")
    
    return tokenizer

# 단어 사전 크기 설정
SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = 20000

# 전처리된 데이터 저장
eng_corpus = []
kor_corpus = []

# 전처리된 문장을 리스트에 저장
for k, e in cleaned_corpus:
    kor_corpus.append(preprocess_sentence(k))
    eng_corpus.append(preprocess_sentence(e))

# 토크나이저 학습
ko_tokenizer = generate_tokenizer(kor_corpus, SRC_VOCAB_SIZE, "ko")
en_tokenizer = generate_tokenizer(eng_corpus, TGT_VOCAB_SIZE, "en")
en_tokenizer.set_encode_extra_options("bos:eos")

# PyTorch에서 바로 사용할 수 있도록 Tensor 변환
def tokenize_and_convert_to_tensor(sentences, tokenizer, max_len=50):
    """
    주어진 문장을 토크나이저로 변환하고, PyTorch Tensor로 변환하는 함수
    """
    tokenized_sentences = [tokenizer.EncodeAsIds(sentence)[:max_len] for sentence in sentences]
    
    # 패딩 처리 (최대 길이 맞추기)
    padded_sentences = [s + [0] * (max_len - len(s)) for s in tokenized_sentences]
    
    # PyTorch Tensor 변환
    return torch.tensor(padded_sentences, dtype=torch.long)

# PyTorch Tensor 변환
kor_tensor = tokenize_and_convert_to_tensor(kor_corpus, ko_tokenizer)
eng_tensor = tokenize_and_convert_to_tensor(eng_corpus, en_tokenizer)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ko_corpus.txt
  input_format: 
  model_prefix: ko
  model_type: UNIGRAM
  vocab_size: 20000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differenti

In [6]:
# 문장 정제 및 토큰화
eng_corpus = []
kor_corpus = []

for k, e in cleaned_corpus:
    kor_corpus.append(preprocess_sentence(k))
    eng_corpus.append(preprocess_sentence(e))

# SentencePiece 토크나이저 학습
ko_tokenizer = generate_tokenizer(kor_corpus, SRC_VOCAB_SIZE, "ko")
en_tokenizer = generate_tokenizer(eng_corpus, TGT_VOCAB_SIZE, "en")
en_tokenizer.set_encode_extra_options("bos:eos")

src_corpus = []
tgt_corpus = []

assert len(kor_corpus) == len(eng_corpus)

# 토큰 길이가 50 이하인 문장만 필터링
for idx in tqdm(range(len(kor_corpus))):
    src_tokens = ko_tokenizer.EncodeAsIds(kor_corpus[idx])
    tgt_tokens = en_tokenizer.EncodeAsIds(eng_corpus[idx])
    
    if len(src_tokens) <= 50 and len(tgt_tokens) <= 50:
        src_corpus.append(torch.tensor(src_tokens, dtype=torch.long))
        tgt_corpus.append(torch.tensor(tgt_tokens, dtype=torch.long))

# PyTorch에서 패딩 적용 (post-padding)
enc_train = pad_sequence(src_corpus, batch_first=True, padding_value=0)
dec_train = pad_sequence(tgt_corpus, batch_first=True, padding_value=0)

# 데이터 확인
print("enc_train shape:", enc_train.shape)  # (batch_size, max_seq_len)
print("dec_train shape:", dec_train.shape)  # (batch_size, max_seq_len)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ko_corpus.txt
  input_format: 
  model_prefix: ko
  model_type: UNIGRAM
  vocab_size: 20000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differenti

  0%|          | 0/78968 [00:00<?, ?it/s]

enc_train shape: torch.Size([73364, 50])
dec_train shape: torch.Size([73364, 50])


In [7]:
### Learning Rate Scheduler (논문 기반 - PyTorch)
class CustomSchedule(optim.lr_scheduler.LambdaLR):
    def __init__(self, optimizer, d_model, warmup_steps=4000, last_epoch=-1):
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        super(CustomSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        """
        학습률 스케줄링 함수 (논문 기반)
        step을 float32로 변환하여 PyTorch에서 오류 방지
        """
        step = max(1, step)  # step이 0이 되는 것을 방지
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        return (self.d_model ** -0.5) * min(arg1, arg2)

# 하이퍼파라미터 설정
d_model = 512
warmup_steps = 4000

# 옵티마이저 설정
model = torch.nn.Linear(d_model, d_model)  # 더미 모델 생성 (옵티마이저 적용을 위해)
optimizer = optim.AdamW(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)

# 학습률 스케줄러 적용
scheduler = CustomSchedule(optimizer, d_model, warmup_steps)

In [8]:
# Positional Encoding: 임베딩에 위치 정보를 추가합니다.
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

# Encoder 레이어 (attention weight 반환)
class TransformerEncoderLayerWithAttn(nn.Module):
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super(TransformerEncoderLayerWithAttn, self).__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, 
                                               dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(d_model, dff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dff, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.activation = nn.ReLU()  # 또는 GELU
        
    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        # Self-attention (attention weight 반환)
        attn_output, attn_weights = self.self_attn(src, src, src, 
                                                   attn_mask=src_mask, 
                                                   key_padding_mask=src_key_padding_mask, 
                                                   need_weights=True)
        src = src + self.dropout1(attn_output)
        src = self.norm1(src)
        
        ff_output = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(ff_output)
        src = self.norm2(src)
        return src, attn_weights

# Decoder 레이어 (self-attention 및 encoder-decoder attention의 weight 반환)
class TransformerDecoderLayerWithAttn(nn.Module):
    def __init__(self, d_model, num_heads, dff, dropout=0.1):
        super(TransformerDecoderLayerWithAttn, self).__init__()
        self.self_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, 
                                               dropout=dropout, batch_first=True)
        self.enc_dec_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, 
                                                  dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(d_model, dff)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dff, d_model)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        
        self.activation = nn.ReLU()
        
    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        # Decoder self-attention
        self_attn_output, self_attn_weights = self.self_attn(tgt, tgt, tgt, 
                                                              attn_mask=tgt_mask, 
                                                              key_padding_mask=tgt_key_padding_mask,
                                                              need_weights=True)
        tgt = tgt + self.dropout1(self_attn_output)
        tgt = self.norm1(tgt)
        
        # Encoder-Decoder attention
        enc_dec_attn_output, enc_dec_attn_weights = self.enc_dec_attn(tgt, memory, memory, 
                                                                      attn_mask=memory_mask,
                                                                      key_padding_mask=memory_key_padding_mask,
                                                                      need_weights=True)
        tgt = tgt + self.dropout2(enc_dec_attn_output)
        tgt = self.norm2(tgt)
        
        # Feed Forward Network
        ff_output = self.linear2(self.dropout(self.activation(self.linear1(tgt))))
        tgt = tgt + self.dropout3(ff_output)
        tgt = self.norm3(tgt)
        return tgt, self_attn_weights, enc_dec_attn_weights

# 전체 Transformer 모델 (Encoder/Decoder를 스택하여 구성)
class TransformerWithAttn(nn.Module):
    def __init__(self, num_layers=2, d_model=512, num_heads=8, dff=2048, 
                 input_vocab_size=20000, target_vocab_size=20000, dropout=0.1):
        super(TransformerWithAttn, self).__init__()
        
        # 임베딩 레이어
        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(target_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, dropout)
        
        # Encoder 레이어들
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayerWithAttn(d_model, num_heads, dff, dropout)
            for _ in range(num_layers)
        ])
        
        # Decoder 레이어들
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderLayerWithAttn(d_model, num_heads, dff, dropout)
            for _ in range(num_layers)
        ])
        
        # 최종 출력 레이어
        self.final_layer = nn.Linear(d_model, target_vocab_size)
    
    def forward(self, src, tgt, src_mask=None, tgt_mask=None, memory_mask=None,
                src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        # 임베딩 및 positional encoding
        src_emb = self.encoder_embedding(src)
        src_emb = self.pos_encoding(src_emb)
        
        tgt_emb = self.decoder_embedding(tgt)
        tgt_emb = self.pos_encoding(tgt_emb)
        
        # Encoder 통과 (각 레이어의 attention weight들을 저장)
        enc_attns = []
        encoder_output = src_emb
        for layer in self.encoder_layers:
            encoder_output, attn_weights = layer(encoder_output, src_mask, src_key_padding_mask)
            enc_attns.append(attn_weights)
        
        # Decoder 통과 (self-attention과 encoder-decoder attention의 weight들을 저장)
        dec_attns = []
        dec_enc_attns = []
        decoder_output = tgt_emb
        for layer in self.decoder_layers:
            decoder_output, self_attn_weights, enc_dec_attn_weights = layer(
                decoder_output, encoder_output, tgt_mask, memory_mask, tgt_key_padding_mask, memory_key_padding_mask)
            dec_attns.append(self_attn_weights)
            dec_enc_attns.append(enc_dec_attn_weights)
        
        transformer_output = decoder_output
        final_output = self.final_layer(transformer_output)
        return final_output, enc_attns, dec_attns, dec_enc_attns

# GPU / MPS / CPU 자동 감지
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 선언 (SRC_VOCAB_SIZE, TGT_VOCAB_SIZE 는 어휘 크기에 맞게 정의)
transformer = TransformerWithAttn(num_layers=2, d_model=512, num_heads=8, dff=2048, 
                                  input_vocab_size=SRC_VOCAB_SIZE, target_vocab_size=TGT_VOCAB_SIZE, dropout=0.1).to(device)

Using device: mps


In [9]:
### 손실 함수 및 마스크 생성 함수 정의 (PyTorch)

# 손실 함수 정의
criterion = nn.CrossEntropyLoss(ignore_index=0)  # PAD 토큰(0) 무시

def loss_function(real, pred):
    """
    손실 함수 정의 (마스킹 처리 포함)
    real: 실제 정답 레이블
    pred: 모델의 예측 값
    """
    loss = criterion(pred.view(-1, pred.shape[-1]), real.view(-1))
    return loss

# 마스크 생성 함수
def create_padding_mask(seq):
    """
    패딩 마스크 생성 함수
    입력: 시퀀스 (batch, seq_len)
    출력: 마스크 (batch, 1, seq_len)
    """
    return (seq == 0).unsqueeze(1).unsqueeze(2)  # (batch, 1, 1, seq_len)

def create_look_ahead_mask(size):
    """
    룩어헤드 마스크 생성 함수
    입력: size (시퀀스 길이)
    출력: (size, size) 행렬로 상삼각형 부분을 마스킹
    """
    mask = torch.triu(torch.ones((size, size)), diagonal=1)  # 상삼각행렬
    return mask.masked_fill(mask == 1, float('-inf'))


def generate_masks(src, tgt, num_heads=8):
    """
    PyTorch의 nn.Transformer와 호환되는 마스크 생성 함수
    """
    batch_size = src.shape[0]
    seq_len_src = src.shape[1]
    seq_len_tgt = tgt.shape[1]

    # 인코더 패딩 마스크
    enc_mask = (src == 0).unsqueeze(1).expand(-1, seq_len_src, seq_len_src)  # (batch, seq_len, seq_len)
    dec_enc_mask = (src == 0).unsqueeze(1).expand(-1, seq_len_tgt, seq_len_src)  # (batch, seq_len_tgt, seq_len_src)

    # 2D 룩어헤드 마스크 생성 (nn.Transformer는 내부적으로 이 마스크를 확장합니다)
    dec_mask = torch.triu(torch.ones((seq_len_tgt, seq_len_tgt), device=tgt.device), diagonal=1)
    dec_mask = dec_mask.masked_fill(dec_mask == 1, float('-inf'))

    # enc_mask와 dec_enc_mask는 이미 배치 차원 기준으로 확장
    enc_mask = enc_mask.repeat(num_heads, 1, 1)           # (batch*num_heads, seq_len, seq_len)
    dec_enc_mask = dec_enc_mask.repeat(num_heads, 1, 1)     # (batch*num_heads, seq_len_tgt, seq_len_src)

    return enc_mask, dec_enc_mask, dec_mask

In [10]:
### 학습 과정 정의 (PyTorch)
def train_step(src, tgt, model, optimizer, criterion):
    """
    모델 학습을 위한 한 스텝 정의 (PyTorch 버전)
    src: 인코더 입력
    tgt: 디코더 입력
    model: Transformer 모델
    optimizer: Adam 옵티마이저
    criterion: 손실 함수
    """
    model.train()  # 모델을 학습 모드로 설정
    
    # <BOS> 토큰을 제외한 정답 레이블 사용
    tgt_input = tgt[:, :-1]  # <EOS> 제외한 입력 사용
    gold = tgt[:, 1:].reshape(-1)  # Flatten 처리
    
    # 마스크 생성 (tgt_input 크기에 맞춰 마스크 생성)
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_input)  # tgt_input 사용
    
    # 옵티마이저 초기화
    optimizer.zero_grad()
    
    # 모델 예측 수행
    predictions = model(src, tgt_input, enc_mask, dec_mask, dec_enc_mask)
    
    # 손실 계산 (CrossEntropyLoss expects shape (batch*seq_len, vocab_size))
    loss = criterion(predictions.view(-1, predictions.shape[-1]), gold)
    
    # 역전파 및 가중치 업데이트
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [12]:
### 5. 학습 과정 실행 및 번역 결과 출력 (PyTorch)

# 예시: translate 함수 정의
def translate(sentence, model, src_tokenizer, tgt_tokenizer, device=device, max_length=50):
    model.eval()
    # 입력 문장을 토큰화
    tokens = src_tokenizer.encode(sentence)
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)
    
    # 초기 토큰은 BOS 토큰: SentencePieceProcessor의 메서드를 사용합니다.
    tgt_tokens = [tgt_tokenizer.bos_id()]
    
    for _ in range(max_length):
        tgt_tensor = torch.LongTensor(tgt_tokens).unsqueeze(0).to(device)
        outputs = model(src_tensor, tgt_tensor)
        final_output = outputs[0]  # 최종 예측값 추출
        # 마지막 시점의 토큰 선택
        next_token = final_output.argmax(dim=-1)[:, -1].item()
        tgt_tokens.append(next_token)
        if next_token == tgt_tokenizer.eos_id():
            break
            
    translation = tgt_tokenizer.decode(tgt_tokens)
    return translation

def train_step(src, tgt, model, optimizer, criterion):
    model.train()  # 학습 모드 전환
    
    tgt_input = tgt[:, :-1]  # 마지막 토큰(<EOS>) 제외
    gold = tgt[:, 1:].reshape(-1)  # Flatten 처리
    
    # 마스크 생성 (generate_masks 함수 사용)
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_input)
    
    optimizer.zero_grad()
    
    # 모델 예측 (모델은 튜플을 반환함)
    outputs = model(src, tgt_input, enc_mask, dec_mask, dec_enc_mask)
    predictions = outputs[0]  # final_output 추출
    
    loss = criterion(predictions.view(-1, predictions.shape[-1]), gold)
    loss.backward()
    optimizer.step()
    
    return loss.item()

# 학습 관련 변수 설정
BATCH_SIZE = 128
EPOCHS = 15
loss_history = []  # 각 epoch의 평균 loss 기록

examples = [
    "오바마는 대통령이다.",
    "시민들은 도시 속에 산다.",
    "커피는 필요 없다.",
    "일곱 명의 사망자가 발생했다."
]

best_loss = float('inf')
best_hyperparams = {}
best_translations = []

# PyTorch 학습 루프
for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list, desc=f'Epoch {epoch + 1}')
    
    for batch, idx in enumerate(t):
        batch_src = enc_train[idx:idx+BATCH_SIZE].to(device)
        batch_tgt = dec_train[idx:idx+BATCH_SIZE].to(device)
        
        batch_loss = train_step(batch_src, batch_tgt, transformer, optimizer, criterion)
        total_loss += batch_loss
        t.set_postfix(loss=total_loss / (batch + 1))
    
    avg_loss = total_loss / len(idx_list)
    loss_history.append(avg_loss)
    
    # 예문 번역 실행
    current_translations = []
    for example in examples:
        translation = translate(example, transformer, ko_tokenizer, en_tokenizer)
        current_translations.append(translation)
    
    # 첫 epoch에는 무조건 업데이트, 이후엔 avg_loss가 낮을 때만 업데이트
    if epoch == 0 or avg_loss < best_loss:
        best_loss = avg_loss
        best_hyperparams = {
            "n_layers": 2,
            "d_model": 512,
            "n_heads": 8,
            "d_ff": 2048,
            "dropout": 0.3,
            "warmup_steps": 4000,
            "batch_size": BATCH_SIZE,
            "epoch_at": epoch + 1
        }
        best_translations = current_translations

# 최적 결과 출력
print("\nBest Translations:")
if len(best_translations) < len(examples):
    print("아직 모든 예문에 대한 번역이 저장되지 않았습니다.")
else:
    for i, example in enumerate(examples):
        print(f"> {i+1}. {best_translations[i]}")

print("\nBest Hyperparameters:")
for key, value in best_hyperparams.items():
    print(f"> {key}: {value}")

Epoch 1:   0%|          | 0/574 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/574 [00:00<?, ?it/s]

Epoch 3:   0%|          | 0/574 [00:00<?, ?it/s]

Epoch 4:   0%|          | 0/574 [00:00<?, ?it/s]

Epoch 5:   0%|          | 0/574 [00:00<?, ?it/s]

...

Epoch 12:   0%|          | 0/574 [00:00<?, ?it/s]

Epoch 13:   0%|          | 0/574 [00:00<?, ?it/s]

Epoch 14:   0%|          | 0/574 [00:00<?, ?it/s]

Epoch 15:   0%|          | 0/574 [00:00<?, ?it/s]


Best Translations:
> 1. fish tuckar memoir startsmanagechmity fossemity fossemity fossemity fossemity fossemity fossemity fossemity fossemity fosse engekrevieweddolph reproduce skydiveraliryz tournamenthondanye periodic thor feliciathemwrestlerhondanye periodic thor feliciathemwrestlerhondanye
> 2. fish buoyant fivepointnext buoyant totalitarian ke popularjord unifi allowanceintestin scudately ponder activ kg farmhouse common nutactory shortl perceptionve chairman ruben kinneunder global remainsmov rapper sworn rapper sworn rapper sworn rapper swornstyle steadyhitdaughter lamps activisionzekundershatter swornanami
> 3. precedenthilton loyal festiv noticeunder zemin clevesaur clevesaur tit cleve tung helmand radwan aft parody the popularjord csamulate badly tour malcolmpersonyerkansa flatt demolish zain deb blow hijacktalia buxundershatter dipp hike bless captivfilmmotivatpardnicam debrim
> 4. fish buoyantcalculatnominatrim elevateagency brokerage firaisaur clevesaur cleve mendunder gl

In [15]:
# Attention 시각화 함수
def visualize_attention(src, tgt, enc_attns, dec_attns, dec_enc_attns):
    def draw(data, ax, x="auto", y="auto"):
        import seaborn as sns
        sns.heatmap(data,
                    square=True,
                    vmin=0.0, vmax=1.0,
                    cbar=False, ax=ax,
                    xticklabels=x,
                    yticklabels=y)
    
    # Encoder Attention
    for layer in range(len(enc_attns)):
        num_heads = enc_attns[layer].shape[1]
        fig, axs = plt.subplots(1, num_heads, figsize=(20, 10))
        print("Encoder Layer", layer + 1)
        for h in range(num_heads):
            # enc_attns[layer] shape: (batch, num_heads, src_len, src_len)
            draw(enc_attns[layer][0, h, :len(src), :len(src)], axs[h], x=src, y=src)
        plt.show()
        
    # Decoder Self-Attention
    for layer in range(len(dec_attns)):
        num_heads = dec_attns[layer].shape[1]
        fig, axs = plt.subplots(1, num_heads, figsize=(20, 10))
        print("Decoder Self-Attention Layer", layer + 1)
        for h in range(num_heads):
            # dec_attns[layer] shape: (batch, num_heads, tgt_len, tgt_len)
            draw(dec_attns[layer][0, h, :len(tgt), :len(tgt)], axs[h], x=tgt, y=tgt)
        plt.show()
        
    # Decoder-Encoder Attention
    for layer in range(len(dec_enc_attns)):
        num_heads = dec_enc_attns[layer].shape[1]
        fig, axs = plt.subplots(1, num_heads, figsize=(20, 10))
        print("Decoder-Encoder Attention Layer", layer + 1)
        for h in range(num_heads):
            # dec_enc_attns[layer] shape: (batch, num_heads, tgt_len, src_len)
            draw(dec_enc_attns[layer][0, h, :len(tgt), :len(src)], axs[h], x=src, y=tgt)
        plt.show()

In [16]:
# 번역 생성 함수 (PyTorch 버전)
def evaluate(sentence, model, src_tokenizer, tgt_tokenizer, device=device, max_length=50):
    """
    주어진 문장에 대해 토큰화 후 모델을 사용해 번역을 생성하고,
    attention weight들(logits, enc_attns, dec_attns, dec_enc_attns)를 반환합니다.
    
    model은 forward 시 (logits, enc_attns, dec_attns, dec_enc_attns)를 반환한다고 가정합니다.
    """
    model.eval()
    # 입력 문장 토큰화
    tokens = src_tokenizer.encode(sentence)
    # 만약 tokenizer가 pieces 반환 메서드가 있다면 사용 (없으면 tokens를 문자열로 변환)
    if hasattr(src_tokenizer, 'encode_as_pieces'):
        pieces = src_tokenizer.encode_as_pieces(sentence)
    else:
        pieces = [str(tok) for tok in tokens]
    
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)
    tgt_tokens = [tgt_tokenizer.bos_id()]
    
    enc_attns, dec_attns, dec_enc_attns = None, None, None
    for i in range(max_length):
        tgt_tensor = torch.LongTensor(tgt_tokens).unsqueeze(0).to(device)
        # model이 logits와 attention weight들을 반환한다고 가정
        outputs = model(src_tensor, tgt_tensor)
        logits = outputs[0]
        enc_attns = outputs[1]
        dec_attns = outputs[2]
        dec_enc_attns = outputs[3]
        
        next_token = logits.argmax(dim=-1)[:, -1].item()
        tgt_tokens.append(next_token)
        if next_token == tgt_tokenizer.eos_id():
            break
    result = tgt_tokenizer.decode(tgt_tokens)
    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [17]:
# 번역(translate) 및 Attention 시각화 결합 함수
def translate(sentence, model, src_tokenizer, tgt_tokenizer, device=device, plot_attention=False, max_length=50):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = evaluate(sentence, model, src_tokenizer, tgt_tokenizer, device=device, max_length=max_length)
    
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    
    if plot_attention:
        # tgt 결과를 공백 기준으로 나누어 label로 사용 (필요 시 tokenizer의 정보를 활용하세요)
        tgt_tokens = result.split()
        visualize_attention(pieces, tgt_tokens, enc_attns, dec_attns, dec_enc_attns)

---

## 회고

- Attention 시각화 함수 & 번역(translate) 및 Attention 시각화 결합 함수가 작동은 되나 시각화가 되지 않는 현상 발생 -> 원인 분석 중\

- 1 epoch 당 평균 5분 정도 소요됨 -> 총 15 epoch, 학습 시에 시간이 너무 오래걸리니 데이터 수를 줄이는 방안 고려 필요